# Пример простой корутины

In [1]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print(f'-> coroutine received {x}')

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x000001A225338900>

In [3]:
next(my_coro)

-> coroutine started


In [4]:
try:
    my_coro.send(42)
except StopIteration as ex:
    print(ex)

-> coroutine received 42



А теперь разберем, что происходит в коде выше:

1. Корутина определяется через функцию-генератор, т.е. функцию с конструкцией `yield` в своем теле, которая ничего не возвращает, а напротив, ждет какого-то значения на входе.  
2. Для работы с корутиной необходимо сначала создать генератор, что и делается путем вызова функции `simple_coroutine()`.  
3. Сразу после создания генератор не находится в режиме ожидания - он еще просто не дошел до выполнения конструкции `yield`.  Чтобы он дошел, надо применить функцию `next()`.  
4. После применения метода `.send()` код корутины отрабатывает и возвращается в режим ожидания.  

# Практический пример использования корутин

Разработаем программу, симулирующую работу таксопарка. Каждая машина совершает фиксированное число поездок, задаваемое изначально, после чего возвращается в таксопарк. С каждой машиной происходят следующие события:

- выезд из гаража  
- посадка пассажира  
- высадка пассажира  
- заезд в гараж  

Также каждое событие характеризуется временем его происхождения и машиной, с которой оно произошло. Поэтому сначала реализуем кортеж, описывающий событие:

In [5]:
from collections import namedtuple

Event = namedtuple('Event', ['time', 'proc', 'action'])

А теперь напишем корутину, реализующую последовательность событий, происходящую с машиной:

In [6]:
def taxi_process(ident, trips, start_time=0):
    """
    ident - это идентификатор автомобиля (0, 1, 2, ...); 
    trips - это число поездок;
    start_time - это время выезда из гаража. Ради удобства будем обозначать его целыми числами
    """
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')

Для использования этого функционала мы должны: 
* вызвать корутину, создав тем самым генератор, вызвать его метод `next()`, чтобы перевести генератор в режим ожидания события 'leave garage'  
* отправить в генератор время этого события, после чего событие сгенерируется с данным временем
* отправить в генератор заранее определенное число событий 'pick up passenger' и 'drop off passenger'  
* отправить в генератор событие 'going home'  

На практике это выглядит так:

In [7]:
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [8]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [9]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [10]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [11]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [12]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

Переменная `time` в этом коде необходима для учета времени события на протяжении всего событийного цикла машины. Мы сендим в эту переменную какое-то значение и это значение используется для генерации кортежа `Event` при последующем сенде. То есть, если мы будем осуществлять присваивание этой переменной только первый раз, то в последующие разы она не будет обновляться. Иными словами, каждый раз, когда мы генерируем кортеж `Event`, в нем используется переменная `time`, инициализированная на какой-то из предыдущих строчек кода (на последней строчке кода, где она инициализирована). 

Также, важно понимать, что, когда мы пишем `_.time`, то мы обращаемся к атрибуту `time` последнего сгенерированного объекта, т.е. к кортежу `Event`.

А теперь напишем класс, который будет выполнять симуляцию.

In [26]:
import queue
import random

DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        #Проходимся по всем машинам и посредством применения функции next()
        #у каждой машины активируем событие leave garage, помещая его в нашу PriorityQueue.
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi: ', proc_id, proc_id * '   ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + self.compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            print(f'*** end of simulation time: {self.events.qsize()} event(s) are pending ***')
            
    def compute_duration(self, previous_action):
        if previous_action in ['leave garage', 'drop off passenger']:
            interval = SEARCH_DURATION
        elif previous_action == 'pick up passenger':
            interval = TRIP_DURATION
        elif previous_action == 'going home':
            interval = 1
        else:
            raise ValueError(f'Unknown previous action: {previous_action}')
        return int(random.expovariate(1/interval)) + 1

Самым интересным в классе `Simulator` является метод `run()`. Основное его назначение - формирование и обработка очереди событий. Очередь событий реализована на основе класса `PriorityQueue`. Это сделано по причине того, что время наступления всех событий кроме первого - это случайная величина и легко может получиться, что для машины, расположенной раньше по списку, следующее событие наступит позже, чем для машины позже по списку. При этом мы помним, что у кортежа `Event` самое первое поле - это время его наступления. То есть, когда мы будем складывать эвенты в `PriorityQueue`, они будут автоматически отсортированы по возрастанию времени наступления события. 

Метод `run()` в ходе своей работы сначала переводит все автомобили в режим ожидания самого первого события (leave garage) и сохраняет его в `PriorityQueue`. Далее это событие вытаскивается из этой очереди и о его наступлении сообщается на консоли. После этого в зависимости от текущего события расчитывается время наступления следующего события, которое сендится в машину, с которой произошло текущее событие. Машина возвращает следующее событие, которое сохраняется в `PriorityQueue`.

In [28]:
num_taxis = 3
DEPARTURE_INTERVAL = 5
taxis = {i: taxi_process(ident=i, trips=(i + 1) * 2, start_time=(i * DEPARTURE_INTERVAL)) for i in range(num_taxis)}
sim = Simulator(taxis)
sim.run(180)

taxi:  0  Event(time=0, proc=0, action='leave garage')
taxi:  0  Event(time=3, proc=0, action='pick up passenger')
taxi:  1     Event(time=5, proc=1, action='leave garage')
taxi:  1     Event(time=7, proc=1, action='pick up passenger')
taxi:  1     Event(time=8, proc=1, action='drop off passenger')
taxi:  2        Event(time=10, proc=2, action='leave garage')
taxi:  2        Event(time=11, proc=2, action='pick up passenger')
taxi:  1     Event(time=13, proc=1, action='pick up passenger')
taxi:  2        Event(time=34, proc=2, action='drop off passenger')
taxi:  2        Event(time=35, proc=2, action='pick up passenger')
taxi:  0  Event(time=37, proc=0, action='drop off passenger')
taxi:  0  Event(time=38, proc=0, action='pick up passenger')
taxi:  1     Event(time=46, proc=1, action='drop off passenger')
taxi:  1     Event(time=47, proc=1, action='pick up passenger')
taxi:  2        Event(time=50, proc=2, action='drop off passenger')
taxi:  2        Event(time=53, proc=2, action='pick 